In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing_extensions import List, TypedDict
from langchain_core.documents import Document
from langchain import hub
from langgraph.graph import START, StateGraph



load_dotenv()

openrouter_key = os.getenv("OPENROUTER_API_KEY")
openrouter_base_url = os.getenv("OPENAI_BASE_URL")

In [2]:
print(openrouter_key)

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


In [7]:
llm = ChatOpenAI(
    model_name="deepseek/deepseek-chat-v3.1:free",
    api_key=openrouter_key,
    base_url=openrouter_base_url
)


emb = HuggingFaceEmbeddings(
    model_name="BAAI/bge-small-en-v1.5",
    encode_kwargs={"normalize_embeddings": True}  # normalize for cosine similarity
)


vector_store = InMemoryVectorStore(emb)

f:\AI_Agent\Graphenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
f:\AI_Agent\Graphenv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\user\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: 

In [5]:
print(llm.invoke("Hello, LangGraph + OpenRouter!").content)

Hello! It looks like you're excited about using **LangGraph** with **OpenRouter**. 

**LangGraph** is a library for building stateful, multi-actor applications with LLMs (like agents, chatbots, and workflows), while **OpenRouter** is a unified API that lets you access multiple LLMs (such as GPT-4, Claude, Llama, etc.) through a single endpoint.

### How can I help you today?
1. **Setting up LangGraph with OpenRouter** – Need guidance on integrating OpenRouter as your LLM provider?
2. **Building a specific workflow** – Creating a chatbot, agent, or custom stateful application?
3. **Debugging an issue** – Running into problems with your graph or OpenRouter API calls?
4. **Something else?** Let me know!

Feel free to share more details, and I'll assist you! 🚀


In [8]:
pdf_path = r"C:\Users\user\Desktop\Khawaja-Bilal-Ahmad.pdf"
loader = PyPDFLoader(pdf_path)
docs = loader.load()
print(f"Loaded {len(docs)} pages")
print(docs[0].page_content[:300])

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(docs)

_ = vector_store.add_documents(documents=all_splits)

Loaded 1 pages
Khawaja Bilal Ahmad
bilalahmad101325@gmail.com
 
Islamabad
 
03365796899
 
03315886499
 
EDUCATION
2022 – present Riphah International University, Bachelors in Computer Science
CAREER OBJECTIVE
Motivated to grow as a developer by building smart, efficient systems that solve real-world problems. 
Eag


In [9]:
prompt = hub.pull("rlm/rag-prompt")

class State(TypedDict):
    question: str
    context: List[Document]
    answer: str



def retrieve(state: State):
    retrieved_docs = vector_store.similarity_search(state["question"])
    return{"context": retrieved_docs}


def generate(state: State):
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt.invoke({"question": state["question"], "context": docs_content})
    response = llm.invoke(messages)
    return {"answer": response.content}


In [10]:
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()



In [20]:
response = graph.invoke({"question": "provide me his skills?"})
print(response["answer"])

Khawaja Bilal Ahmad's skills include Python, ReactJS, and TailwindCSS for languages and front-end development. He is skilled in AI/ML areas such as fine-tuning LLMs, using Hugging Face models, and feature engineering. His software development skills include Git/GitHub, FastAPI, SDLC, and Agile methodologies.


In [18]:
print(f"Context: {response['context']}\n\n")
print(f"Answer: {response['answer']}")

Context: [Document(id='db30645a-2b2f-4dd2-9753-7be1dcfe6ebb', metadata={'producer': 'Skia/PDF m135', 'creator': 'FlowCV - https://flowcv.com', 'creationdate': '2025-06-15T11:07:22+00:00', 'moddate': '2025-06-15T11:07:22+00:00', 'keywords': 'FlowCV – Online Resume Builder – https://flowcv.com', 'source': 'C:\\Users\\user\\Desktop\\Khawaja-Bilal-Ahmad.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1'}, page_content='Khawaja Bilal Ahmad\nbilalahmad101325@gmail.com\n \nIslamabad\n \n03365796899\n \n03315886499\n \nEDUCATION\n2022 – present Riphah International University, Bachelors in Computer Science\nCAREER OBJECTIVE\nMotivated to grow as a developer by building smart, efficient systems that solve real-world problems. \nEager to apply my skills in AI, backend development, and software engineering while continuously \nlearning and contributing to meaningful tech solutions.\nEXPERIENCE\nAI Integration Specialist\nAbsouilt IT Initiative (2024, 6 weeks)\n•Automated a taxi dispatch system 